# Identify Flares/Stars with coinciding SPT and TESS observations
This notebook searches TESS for the flaring stars in the SPT flare catalog (Tandoi et.al. 24) and identifies sectors of coinciding observation.
The identifying data of the stars with coinciding observations are saved to `spt_tess_candidates.pkl`.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import pandas as pd

from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

from scipy.linalg import lstsq
from scipy.optimize import curve_fit

import lightkurve as lk
import tessreduce as tr

import pickle

c:\Users\vishn\miniconda3\envs\SPT_Flares\lib\site-packages\lightkurve\config\__init__.py:119: UserWarning: The default Lightkurve cache directory, used by download(), etc., has been moved to C:\Users\vishn\.lightkurve\cache. Please move all the files in the legacy directory C:\Users\vishn\.lightkurve-cache to the new location and remove the legacy directory. Refer to https://docs.lightkurve.org/reference/config.html#default-cache-directory-migration for more information.
  warnings.warn(


<span style="color:red; font-weight:bold;"> See these tools for relevant tess sector identification:</span> 
- TESS-point: https://github.com/tessgi/tess-point
  - This would greatly restrict sectors we search for each star, so worth exploring for very large catalogs
- TESS-point Web Tool: https://heasarc.gsfc.nasa.gov/wsgi-scripts/TESS/TESS-point_Web_Tool/TESS-point_Web_Tool/wtv_v2.0.py/

First we retrieve the flare locations and times for SPT and TESS orbit details

The SPT "Flare Star Catalog" is retrieved from https://pole.uchicago.edu/public/data/tandoi24/#Overview.

TESS sector orbit times are retrieved from https://tess.mit.edu/observations/.

In [2]:
spt_flares_df = pd.read_csv('..\data\SPT\online_pipeline_flagged_stars_12jun2025.csv')
spt_flares_df.sort_values(by= 'mjd')
spt_flares_df['sectors'] = [[] for _ in range(len(spt_flares_df))]
print(spt_flares_df.columns)
# print(spt_flares_df[['ra', 'dec']].head())


TESS_sectors_df = pd.read_csv("..\data\TESS\TESS_orbit_times.csv")
# Times are stored by orbit, so first merge by sector
TESS_sectors_df = TESS_sectors_df.groupby("Sector").agg({
    "Start of Orbit": "min",
    "End of Orbit": "max"
}).reset_index()
TESS_sectors_df.columns = ["Sector", "Sector Start", "Sector End"]

# Formatting to MJD for comparison
TESS_sectors_df = TESS_sectors_df.dropna(subset=['Sector Start'])
TESS_sectors_df = TESS_sectors_df.dropna(subset=['Sector Start'])
TESS_sectors_df['Sector'] = TESS_sectors_df['Sector'].apply(lambda x: int(x))
TESS_sectors_df['Sector Start'] = TESS_sectors_df['Sector Start'].apply(lambda x: Time(x, format= 'iso').utc.mjd)
TESS_sectors_df['Sector End'] = TESS_sectors_df['Sector End'].apply(lambda x: Time(x, format= 'iso').utc.mjd)
print(TESS_sectors_df.columns)

Index(['ra', 'dec', 'obsid', 'mjd', 'sectors'], dtype='object')
Index(['Sector', 'Sector Start', 'Sector End'], dtype='object')


Using this data we find the range of TESS sectors over which SPT observed the flares. This is not precise and simply cuts out sectors before the first flare and after the last flare in the catalog.

In [3]:
spt_t_bounds = (spt_flares_df.iloc[0]['mjd'], spt_flares_df.iloc[-1]['mjd'])
t_bounds = Time(spt_t_bounds, format= 'mjd')
print(t_bounds.iso)

# Look for TESS Sectors between first flare start and last flare end
valid_sectors = TESS_sectors_df.loc[(TESS_sectors_df['Sector Start'] <= t_bounds[1].value) & (TESS_sectors_df['Sector End'] >= t_bounds[0].value)]
valid_sectors = set(valid_sectors['Sector'].apply(lambda x: int(x)))
print(valid_sectors)

['2021-05-04 15:34:23.000' '2025-06-08 13:30:24.000']
{38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93}


For each flare in spt_flare_catalog first grab FFIs for all sectors that contain its skycoordinates using tesscut. Lightkurve search is generally fast and I suspect may require less local runtime than checking if flare falls withing explicit time range of each sector.

Then, for the sectors where TESS is looking at the star and check the sector times to identify the one with the flare.

In [4]:
for i, flare in spt_flares_df.iterrows():
    ra = flare['ra']
    dec = flare['dec']
    flare_obs_start = Time(flare['mjd'],  format='mjd', scale='utc')
    c = SkyCoord(ra, dec, unit= 'deg')

    # Search for any sectors where TESS is looking at the star
    possible_matches = lk.search_tesscut(c, [sector for sector in valid_sectors])
    if len(possible_matches) >= 1:
        # Check time range for each sector
        for data_prod in possible_matches:
            sector = int(data_prod.mission[0][11:])
            sector_time = TESS_sectors_df[TESS_sectors_df['Sector'] == sector]
            sector_start = sector_time['Sector Start'].iloc[0]
            sector_end = sector_time['Sector End'].iloc[0]
            if sector_start < flare_obs_start.value < sector_end:
                print(f'{i}/{spt_flares_df.shape[0]}: Match found for Sector {sector}')
                flare['sectors'] = flare['sectors'].append(sector)
            else:
                print(f'{i}/{spt_flares_df.shape[0]}: Sector {sector} - no overlap in time')
    else:
        print(f'{i}/{spt_flares_df.shape[0]}: no nearby targets')


0/39: Sector 68 - no overlap in time
0/39: Sector 69 - no overlap in time
1/39: Sector 67 - no overlap in time
2/39: Sector 69 - no overlap in time
3/39: Sector 68 - no overlap in time
3/39: Sector 67 - no overlap in time
4/39: Sector 68 - no overlap in time
5/39: Sector 68 - no overlap in time
5/39: Sector 69 - no overlap in time
6/39: Sector 69 - no overlap in time


No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (88.36147, -43.55822)>".


7/39: no nearby targets


No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (73.707, -29.5655)>".


8/39: no nearby targets
9/39: Sector 61 - no overlap in time
9/39: Sector 87 - no overlap in time
9/39: Sector 88 - no overlap in time
10/39: Sector 67 - no overlap in time
11/39: Sector 69 - no overlap in time
12/39: Sector 68 - no overlap in time
12/39: Sector 69 - no overlap in time
13/39: Sector 69 - no overlap in time
14/39: Sector 68 - no overlap in time
15/39: Sector 87 - no overlap in time
16/39: Sector 68 - no overlap in time
16/39: Sector 67 - no overlap in time
16/39: Sector 69 - no overlap in time
17/39: Sector 68 - no overlap in time
18/39: Sector 67 - no overlap in time
19/39: Sector 69 - no overlap in time
20/39: Sector 69 - no overlap in time
21/39: Sector 67 - no overlap in time


...

22/39: no nearby targets
23/39: Sector 69 - no overlap in time


No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (48.11014, -44.423)>".


24/39: no nearby targets
25/39: Sector 68 - no overlap in time
26/39: Sector 68 - no overlap in time
26/39: Sector 69 - no overlap in time
27/39: Sector 69 - no overlap in time
28/39: Sector 67 - no overlap in time
29/39: Sector 69 - no overlap in time
30/39: Sector 61 - no overlap in time
30/39: Sector 87 - no overlap in time
30/39: Sector 88 - no overlap in time
31/39: Sector 65 - no overlap in time
31/39: Sector 69 - no overlap in time
31/39: Sector 62 - no overlap in time
31/39: Match found for Sector 89
32/39: Sector 67 - no overlap in time
33/39: Sector 67 - no overlap in time
34/39: Sector 69 - no overlap in time
35/39: Sector 68 - no overlap in time
35/39: Sector 67 - no overlap in time


No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (42.79707, -47.88438)>".


36/39: no nearby targets
37/39: Sector 69 - no overlap in time
38/39: Sector 68 - no overlap in time


In [5]:
spt_tess_candidates = spt_flares_df[spt_flares_df['sectors'].apply(lambda x: len(x) > 0)].reset_index()
spt_tess_candidates

,index,ra,dec,obsid,mjd,sectors
0,31,59.92104,-58.67446,256943755,60727.886053,[89]


Print out flare skycoord in hmsdms to search in SINBAD and find common identifier for flares; I chose TIC.

In [6]:

for i, flare in spt_tess_candidates.iterrows():
    ra = flare['ra']
    dec = flare['dec']
    c = SkyCoord(ra, dec, frame= 'icrs', unit= 'deg')
    print(i, c.to_string('hmsdms'))

0 03h59m41.0496s -58d40m28.056s


Add TIC to dataframe

In [9]:
spt_tess_candidates['TIC'] = spt_tess_candidates['obsid']
spt_tess_candidates['dr3_source_id'] = spt_tess_candidates['obsid']

spt_tess_candidates.at[0,'TIC'] = "TIC198006444"
spt_tess_candidates.at[0,'dr3_source_id'] = "4682624554442312576"

spt_tess_candidates

,index,ra,dec,obsid,mjd,sectors,TIC,dr3_source_id
0,31,59.92104,-58.67446,256943755,60727.886053,[89],TIC198006444,4682624554442312576


In [14]:
Time(spt_tess_candidates.at[0,'mjd'], format='mjd', scale='tdb').utc.iso

'2025-02-21 21:14:45.815'

Save dataframe

In [15]:
spt_tess_candidates.to_pickle('..\data\spt_tess_candidates_online_unverified.pkl')